In [1]:
# Necessary Import Statements
import pandas as pd
import numpy as np

# Datasets converted to Dataframes
#BetaData = pd.read_csv('~/ASDRP/Data/BetaData.csv', low_memory=False)
#data = pd.read_csv('~/ASDRP/Data/GammaData.csv', low_memory=False)
data = pd.read_csv('PS_2021.01.18_09.15.37.csv', low_memory=False)

# Initializing dataframes
duplicates = pd.DataFrame(columns=list(data.columns))
deDupedData = pd.DataFrame(columns=list(data.columns))

# Removing entries with no dates
data.dropna(how='any', subset=['pl_pubdate'], inplace=True)

# Removing entries with bad dates
for index, row in data.iterrows():
    date = data['pl_pubdate'][index]
    try:
        if len(date) < 8:
            data['pl_pubdate'][index] = date + '-01'
        data['pl_pubdate'][index] = pd.to_datetime(data['pl_pubdate'][index]) # This line is redundant, remove either this or line 56
    except:
        data.drop([index], inplace=True)
        data.reset_index(drop=True)

# Converting dates to datetime objects
data['pl_pubdate'] = pd.to_datetime(data['pl_pubdate'])

# Sorting the data
sortedData = data.sort_values(by=['pl_name', 'pl_pubdate'], ascending= [1,0], inplace=False, na_position='last')

# Store the duplicates before removing them
duplicates = pd.DataFrame(columns=list(data.columns))
mask = sortedData.duplicated(subset='pl_name', keep='first')
df_keep = sortedData.loc[~mask]
duplicates = duplicates.append(sortedData.loc[mask])

# Dropping the duplicates
deDupedData = sortedData.drop_duplicates(subset = 'pl_name', keep='first', inplace=False, ignore_index=True)

# Printing the first 10 entries of the DeDupedData
print(deDupedData[['pl_name', 'pl_pubdate']][0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                   pl_name pl_pubdate
0                 11 Com b 2011-08-01
1                 11 UMi b 2017-03-01
2                 14 And b 2011-08-01
3                 14 Her b 2017-03-01
4               16 Cyg B b 2017-03-01
5                 18 Del b 2011-08-01
6  1RXS J160929.1-210524 b 2015-03-01
7                 24 Boo b 2018-08-01
8                 24 Sex b 2011-01-01
9                 24 Sex c 2011-01-01


In [ ]:
# Imports the datasets
phl = pd.read_csv('PHL_Dataset.csv', low_memory=False)

In [ ]:
# Removes any planets that have an NA eccentricity value
deDupedData = deDupedData[deDupedData['pl_orbeccen'].notna()]
deDupedData = deDupedData[deDupedData['pl_rade'].notna()]

# Resets the data row index
deDupedData = deDupedData.reset_index (drop = True)

In [ ]:
# Makes an empty m_type dataframe with same column headers
g_type = pd.DataFrame(columns=list(data.columns))

# Makes an empty m_type_habitable dataframe to store planets that result out of this code
g_type_habitable = pd.DataFrame(columns=list(data.columns))

# Fills the m_type dataframe with M-type host stars' planets
g_type = deDupedData.loc[(deDupedData['st_spectype'].str.startswith('G', na=False))]

# Resets the m_type row index
g_type = g_type.reset_index (drop = True)

In [ ]:
# Takes out planets that have no planet radius and eccentricity
phl = phl[phl['pl_rade'].notna()]
phl = phl[phl['pl_orbeccen'].notna()]

# Resets index
phl = phl.reset_index (drop = True)

In [ ]:
# Makes a new empty dataframe for the m-type planets in the PHL dataset
phl_g_type = pd.DataFrame(columns=list(phl.columns))

In [ ]:
# Fills the empty dataframe with m-type planets from the PHL dataset
phl_g_type = phl.loc[(phl['st_spectype'].str.startswith('G', na=False))]

# Resets index
phl_g_type = phl_g_type.reset_index (drop = True)

In [ ]:
def check_match(planet, control_planet):
    if pd.isna(planet['pl_dens']):
        if (not pd.isna(planet['pl_radj'])) & (not pd.isna(planet['pl_massj'])):
            planet['pl_dens'] = (planet['pl_massj'] * (1.898 * (10 ** 27) * 1000)) \
            / (4 * np.pi * ((planet['pl_radj'] * 43441 * 160934) ** 3) / 3)
        else:
            return
        
    if pd.isna(planet['pl_orbsmax']) and (pd.isna(planet['pl_ratdor']) or pd.isna(planet['st_rad'])):
        return

    # Calculates orbital period in days for main planet
    if pd.isna(planet['pl_orbper']):
        if pd.isna(planet['pl_ratdor']):
            planet['pl_orbper'] = np.sqrt(planet['pl_orbsmax']**3) * 365
        elif not pd.isna(planet['st_rad']): 
            planet['pl_orbper'] = np.sqrt((planet['pl_ratdor']*planet['st_rad'])**3) * 365
        if pd.isna(planet['pl_orbper']):
            return
    
    if pd.isna(control_planet['pl_orbsmax']) and (pd.isna(control_planet['pl_ratdor']) or pd.isna(control_planet['st_rad'])):
        return

    # Calculates orbital period in days for phl planet
    if pd.isna(control_planet['pl_orbper']):
        if pd.isna(control_planet['pl_ratdor']):
            control_planet['pl_orbper'] = np.sqrt(control_planet['pl_orbsmax']**3) * 365
        elif not pd.isna(control_planet['st_rad']): 
            control_planet['pl_orbper'] = np.sqrt((control_planet['pl_ratdor']*control_planet['st_rad'])**3) * 365
        if pd.isna(control_planet['pl_orbper']):
            return
    
    if pd.isna(planet['pl_orbeccen']):
        return
    

    if (float(planet['pl_dens']) >= 4) & (float(planet['pl_dens']) <= 7):
        if (planet['pl_controv_flag'] == '0'):
            if ('Transit' in planet['discoverymethod']):
                if (planet['pl_orbeccen'] <= 0.2):
                    if (np.abs(float(planet['pl_orbper']) - float(control_planet['pl_orbper'])) <= 50):
                        global g_type_habitable
                        g_type_habitable = m_type_habitable.append(planet, ignore_index = True)


In [ ]:
for i in range (len(g_type['pl_rade'])): 
    current_dist = np.abs(phl_g_type['pl_rade'][0]-g_type['pl_rade'][i])
    if current_dist < 20:
        planet = g_type.loc[i].copy()
        control_planet = phl_g_type.loc[0].copy()
        check_match(planet, control_planet)

In [ ]:
g_type_habitable

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292


In [ ]:
phl_g_type

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289
0,14532,Kepler-22 b,Kepler-22,b,NaN,NaN,TIC 158984573,Gaia DR2 2127941757262806656,1,1,1,0,0,Transit,2011,<a refstr=BORUCKI_ET_AL__2012 href=https://ui....,2012-02,Space,Kepler,0.95 m Kepler Telescope,Kepler CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=BORUCKI_ET_AL__2012 href=https://ui....,289.8623,0.0016,-0.002,0.0,0.849,0.018,...,NaN,NaN,NaN,NaN,10.115,0.023,-0.023,10.145,0.02,-0.02,10.1,0.042,-0.042,9.081,NaN,NaN,11.6444,0.000239,-0.000239,NaN,NaN,NaN,11.1894,0.006,-0.006,11.664,NaN,NaN,2014-05-14,2012-02,2014-05-14,1,0,0,0,0,2,NaN,NaN,NaN


In [ ]:
phl_g_type['pl_orbper']

0    289.8623
Name: pl_orbper, dtype: object

In [ ]:
g_type.head()

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,132,BD+20 594 b,BD+20 594,b,NaN,NaN,NaN,NaN,0,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,2016-10,Space,K2,0.95 m Kepler Telescope,Kepler CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,41.6855,0.003,-0.0031,0,0.2410,0.0190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.04,NaN,NaN,2018-04-25,2016-10,2016-07-28,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,156,CoRoT-1 b,CoRoT-1,b,NaN,NaN,NaN,NaN,1,1,1,0,0,Transit,2008,<a refstr=BARGE_ET_AL__2008 href=https://ui.ad...,2008-05,Space,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,1,0,0,1,0,1,0,0,0,0,Published Confirmed,0,<a refstr=BARGE_ET_AL__2008 href=https://ui.ad...,1.5089557,6.4E-06,-6.4E-06,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-14,2008-05,2014-05-14,0,6,1,0,5,20,NaN,NaN,NaN,NaN,NaN,NaN
2,174,CoRoT-13 b,CoRoT-13,b,NaN,NaN,NaN,NaN,1,1,1,0,0,Transit,2010,<a refstr=CABRERA_ET_AL__2010 href=https://ui....,2010-11,Space,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=CABRERA_ET_AL__2010 href=https://ui....,4.03519,3E-05,-3E-05,0,0.0510,0.0031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-14,2010-11,2014-05-14,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,180,CoRoT-16 b,CoRoT-16,b,NaN,NaN,NaN,NaN,1,1,1,0,0,Transit,2011,<a refstr=OLLIVIER_ET_AL__2012 href=https://ui...,2012-05,Space,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=OLLIVIER_ET_AL__2012 href=https://ui...,5.35227,0.0002,-0.0002,0,0.0618,0.0015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-14,2012-05,2014-05-14,0,0,2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,182,CoRoT-17 b,CoRoT-17,b,NaN,NaN,NaN,NaN,1,1,1,0,0,Transit,2011,<a refstr=CSIZMADIA_ET_AL__2011 href=https://u...,2011-07,Space,CoRoT,0.27 m CoRoT Telescope,CoRoT CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=CSIZMADIA_ET_AL__2011 href=https://u...,3.7681,0.0003,-0.0003,0,0.0461,0.0008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-14,2011-07,2014-05-14,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
